### Load 15m Links

In [1]:
import csv

load_links = []

with open('fashion_15m_id.csv', 'r') as file:
    reader = csv.reader(file, delimiter=',', quotechar='|')
    for row in reader:
        load_links.append(row)
    file.closed
    
load_links = load_links[0]

In [2]:
len(load_links)

151242

In [3]:
load_links[0]

'"BtBaeptHn8Z"'

In [4]:
load_links = [w.replace('\"', '') for w in load_links]
load_links[:10]

['BtBaeptHn8Z',
 'BtBmwyHlOZI',
 'BtBQgi6BoNY',
 'BtBhggYg2Cf',
 'Bs5k9YQBDxd',
 'BssV3vxB9Od',
 'BtBBjWGg1lq',
 'BsuoYFghY0A',
 'BspHbGGh74o',
 'Bs5if_6hh-6']

### Get Rawdata

In [5]:
import os
import time
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
import requests
import re
import json
import emoji
from bs4 import BeautifulSoup

def data2json(id, username, date, hashtags, final_image_link, likes_num, comments_num):
    single_data = {
        "id" : id,
        "username" : username,
        "date" : date,
        "hashtags" : hashtags,
        "imagelinks" : final_image_link,
        "likes" : likes_num,
        "comments" : comments_num
    }
    return single_data

def save_json_file(my_tag, myjson):
    file_name = my_tag + '_rawdata.json'
    with open(file_name, 'w') as file:
        json.dump(myjson, file)

def crawling_rawdata_from_list(file_name, my_links):
    json_list = []
    emoji_keys = emoji.UNICODE_EMOJI.keys()
    
    for i in range(len(my_links)):
        url = 'https://instagram.com/p/' + my_links[i]
        req = requests.get(url)
        html = req.text
        header = req.headers
        status = req.status_code
        soup = BeautifulSoup(html, 'html.parser')

        script_contents = soup.find_all('script')
        
        ## id
        id = my_links[i]
        
        ## username
        user_content = soup.find(rel = "canonical")
        if not user_content:
            username = ''
        else:
            username_href = user_content.get('href')
            username_p = re.compile("instagram.com/(.*)/p")
            username = username_p.findall(str(username_href))
            if len(username) != 0:
                username = username[0]
            else:
                username = ''

        ## date
        date_p = re.compile("uploadDate\":\"(.{19})\"")
        date = date_p.findall(str(script_contents))
        if len(date) != 0:
            date = date[0]
        else:
            date = ''

        ## hashtags
        meta_content = soup.find_all(property = "instapp:hashtags")
        hashtags = []    
        if meta_content:
            hash_tags_p = re.compile("content=\"(.*?)\"")
            emoji_hashtag = hash_tags_p.findall(str(meta_content))
            
            for tag in emoji_hashtag:
                for e in emoji_keys:
                    emoji_have = tag.find(e)
                    if emoji_have > -1:
                        tag = tag.replace(e, '')
                hashtags.append(tag)
            hashtags = list(filter(None, hashtags))

        ## image links
        image_links_p = re.compile("\"display_resources\":\[.*?\]")
        image_links = image_links_p.findall(str(script_contents))
        
        final_image_link = []
        single_image_link_p = re.compile("\"src\":\"(.*?)\"")
        for dif_img in image_links:
            link = single_image_link_p.findall(str(dif_img))[2]
            final_image_link.append(link)
            
        if len(final_image_link) != 1:
            final_image_link = final_image_link[1:]   
            
        ## likes num
        likes_num_p = re.compile("\"description\":\"(.*?)Likes")
        likes_num = likes_num_p.findall(str(script_contents))

        if len(likes_num) > 0:
            likes_num = likes_num[0]
            likes_num = re.sub('[,\s]', '', likes_num)
        else:
            likes_num = '0'
        
        ## comments num
        comments_num_p = re.compile("\"description\":\".*?,(.*?)Comments")
        comments_num = comments_num_p.findall(str(script_contents))

        if len(comments_num) > 0:
            comments_num = comments_num[0]
            comments_num = re.sub('[,\s]', '', comments_num)
        else:
            comments_num = '0'
        
        ## save data as json
        single_json = data2json(id, username, date, hashtags, final_image_link, likes_num, comments_num)
        json_list.append(single_json)
        time.sleep(1)
        if i % 10 == 0:
            print(i, '번째 데이터')
            print(single_json)
            print('------------------')

    save_json_file(file_name, json_list)

In [6]:
len(load_links)

151242

In [7]:
if 757 in skip:
    print('in')

NameError: name 'skip' is not defined

In [ ]:
## divide test

divide = 200
size = round(len(load_links)/divide)
next_start = 0
next_end = size

skip = [1]
skip = list(map(lambda x : x*size, skip))

for i in range(divide):
    if next_end in skip:
        next_start = next_end + 1
        next_end = next_end + size
        continue
        
    if len(load_links) < next_end:
        next_end = len(load_links) - 1
    print(next_start, next_end)
    
    next_start = next_end + 1
    next_end = next_end + size

In [ ]:
divide = 200
size = round(len(load_links)/divide)
next_start = 1 * size
next_end = size

skip = [1, 2]
skip = list(map(lambda x : x*size, skip))

for i in range(divide):
    try:
        if next_end in skip:
            next_start = next_end + 1
            next_end = next_end + size
            continue

        if len(load_links) < next_end:
            next_end = len(load_links) - 1

        divide_links = load_links[next_start : next_end]
        filename = 'fashion' + str(next_start) + 'to' + str(next_end)

        crawling_rawdata_from_list(filename, divide_links)

        print(next_end, 'done')

        next_start = next_end + 1
        next_end = next_end + size
        
    except:
        pass

0 번째 데이터
{'id': 'BtBJ2hUH-ez', 'username': 'makeup_kingsli_project', 'date': '2019-01-24T12:54:19', 'hashtags': ['визаж', 'съемки', 'школавизажа', 'тренд', 'fashionblogger', 'зеленыегубы', 'влажнаякожа', 'fashion', 'fashionweek', 'макияж'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/eb1156aed9fc533d1b2e951cc80cb140/5D013ED3/t51.2885-15/e35/49682894_538481983300607_1479830252275559133_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': '17', 'comments': '0'}
------------------
10 번째 데이터
{'id': 'Bs5ilLjllFY', 'username': 'dogantextile', 'date': '2019-01-21T13:56:28', 'hashtags': ['fitness', 'uk', 'fitnessgirl', 'fashionista', 'jeans', 'shoes', 'trousers', 'piccadillycircus', 'shawl', 'happy', 'oxfordstreet', 'happynewyear', 'makeup', 'oxford', 'piccadily', 'fashion', 'coventgarden', 'england', 'legging', 'londonfashion', 'leggings', 'unitedkingdom', 'likeforlikes', 'style', 'trouserstyles', 'london', 'portobellomarket', 'londoneye', 'portobelloroad', 'wholesale'], 